# Multi-fidelity (MF) optimization

In most cases it is better to do many cheap evaluations of an approximation to the target function than it is to only optimize the target function. This example demonstrates the 'multi-fidelity' capabilities of xopt. 

We follow the implementation of multi-fidelity bayesian optimization used in botorch https://botorch.org/tutorials/multi_fidelity_bo to optimize the synthetic test function AugmentedHartmann https://botorch.org/api/test_functions.html.

The difference between normal Bayesian optimization and MF optimization is that we specify a 'cost' to making observations at a given fidelity. For this example we assume a base cost of 5 and a fidelity cost between 0-1. The algorithm should make many observations at lower fidelity relative to higher fidelity, lowering the total observation cost. 

NOTE: The cost parameter is required to be the LAST element of the variables list. Also this method is best suited for parallel observations of the test function.

In [1]:
# Import the class
from xopt import Xopt
from xopt.bayesian.generators.multi_fidelity import MultiFidelityGenerator
from xopt.bayesian.models.models import create_multi_fidelity_model
from botorch.test_functions.multi_fidelity import AugmentedHartmann
import logging
import os
SMOKE_TEST = os.environ.get('SMOKE_TEST')

from botorch.acquisition.analytic import UpperConfidenceBound


In [2]:
# To see log messages
from xopt import output_notebook
output_notebook()

The `Xopt` object can be instantiated from a JSON or YAML file, or a dict, with the proper structure.

Here we will make one

In [3]:
# Make a proper input file. 
import yaml
YAML = """
xopt: {output_path: null}

algorithm:
  name: multi_fidelity
  options:  
      processes: 4
      budget: 32
      generator_options: {}

simulation: 
  name: test_multi_fidelity
  evaluate: xopt.tests.evaluators.multi_fidelity.evaluate

vocs:
  description: null
  templates: null
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]                          ## NOTE: THIS IS REQUIRED FOR MULTI-FIDELITY OPTIMIZATION
  objectives:
    y1: 'MINIMIZE'
  linked_variables: {}
  constants: {a: dummy_constant}

"""
config = yaml.safe_load(YAML)

In [4]:
if SMOKE_TEST:
    config['algorithm']['options']['budget'] = 3
    config['algorithm']['options']['processes'] = 1
    config['algorithm']['options']['generator_options']['num_restarts'] = 2
    config['algorithm']['options']['generator_options']['raw_samples'] = 2
    config['algorithm']['options']['generator_options']['base_acq'] = None

X = Xopt(config)
X

Loading config from dict.
Loading config from dict.
Loading config from dict.
Loading config from dict.
Loading config from dict.
`description` keyword no longer allowed in vocs config, removing
`templates` keyword no longer allowed in vocs config, moving to simulation `options`



            Xopt 
________________________________           
Version: 0.4.3+227.g68220e4.dirty
Configured: True
Config as YAML:
xopt: {output_path: null}
algorithm:
  name: multi_fidelity
  options:
    processes: 4
    budget: 32
    generator_options: {}
    base_cost: 1.0
  function: xopt.bayesian.algorithms.multi_fidelity_optimize
simulation:
  name: test_multi_fidelity
  evaluate: xopt.tests.evaluators.multi_fidelity.evaluate
  options: {templates: null, extra_option: abc}
vocs:
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]
  objectives: {y1: MINIMIZE}
  linked_variables: {}
  constants: {a: dummy_constant}
  constraints: null

# Run BayesOpt

In [5]:
# Pick one of these
from concurrent.futures import ThreadPoolExecutor as PoolExecutor
#from concurrent.futures import ProcessPoolExecutor as PoolExecutor

#executor = PoolExecutor()
# This will also work. 
executor=None

In [6]:
# Change max generations
X.run(executor=executor)
results = X.results

Starting at time 2021-10-09T17:40:02-07:00
started running optimization with generator: <xopt.bayesian.generators.multi_fidelity.MultiFidelityGenerator object at 0x129617ee0>
starting optimization loop
Submitted candidate   0, cost: 1.38, total cost: 1.376
Submitted candidate   1, cost: 1.17, total cost: 2.548
Submitted candidate   2, cost: 1.12, total cost: 3.664
Submitted candidate   3, cost: 1.37, total cost: 5.035
generating 4 new candidate(s)
Submitted candidate   4, cost:  1.0, total cost: 6.035
Submitted candidate   5, cost:  1.0, total cost: 7.035
Submitted candidate   6, cost:  1.0, total cost: 8.035
Submitted candidate   7, cost: 1.01, total cost: 9.045
generating 4 new candidate(s)
Submitted candidate   8, cost:  1.0, total cost: 10.05
Submitted candidate   9, cost:  1.0, total cost: 11.05
Submitted candidate  10, cost:  1.0, total cost: 12.05
Submitted candidate  11, cost:  1.0, total cost: 13.05
generating 4 new candidate(s)
Submitted candidate  12, cost:  1.0, total cost:

### Get highest fidelity global optimum

NOTE: the correct global minimum is -3.32237

In [7]:
# create generator object
gen = MultiFidelityGenerator(X.vocs)

In [8]:
# create model
model = create_multi_fidelity_model(results['variables'],
                                    results['corrected_objectives'],
                                    None,
                                    X.vocs)

In [9]:
## NOTE: we want to get the minimum evaluated at the highest fidelity -> make sure to use get_recommendation
rec = gen.get_recommendation(model)
problem = AugmentedHartmann(negate=False)
problem(rec) ## NOTE: the correct global minimum is -3.32237

tensor([-3.0976], dtype=torch.float64)

In [10]:
# Cleanup
!rm results.json